# Path Tracing

In [2]:
import cv2
import numpy as np
# pip install Pillow
    # open Anaconda Prompt and paste above line (without '#') to install package
from PIL import Image

In [3]:
# function that gives a range on hues given a color
def get_limits(color):
    c = np.uint8([[color]])
    hsvC = cv2.cvtColor(c, cv2.COLOR_BGR2HSV)
    
    lowerLimit = hsvC[0][0][0] - 10, 100, 100
    upperLimit = hsvC[0][0][0] + 10, 255, 255
    # the +/-10 defines the range of hues that fall within the limits (the h in hsv)
    # the range on saturation and value is much bigger because we are only looking for hue
    
    lowerLimit = np.array(lowerLimit, dtype=np.uint8)
    upperLimit = np.array(upperLimit, dtype=np.uint8)

    return lowerLimit, upperLimit

In [4]:
color = [0, 255, 255] # color in BGR colorspace

first = False
firstFrame = 0
points = list() # array that will hold locations of detected colors
pointsPrev = list() # array that will hold the locations of detected colors from the previous frame
capture = cv2.VideoCapture(0)
while True:
    ret, frame = capture.read()

    if (first == False): # runs only once, gets the first frame of the webcam video so we can draw the path on it
        firstFrame = frame
        first = True
    
    frame_blur = cv2.GaussianBlur(frame, (11, 11), 17)
    # blurring the image may help get the desired result, but it can be removed
    
    frame_hsv = cv2.cvtColor(frame_blur, cv2.COLOR_BGR2HSV) # convert to HSV

    lowerLimit, upperLimit = get_limits(color) # range of hues that we want the software to detect
    
    mask = cv2.inRange(frame_hsv, lowerLimit, upperLimit) # detects objects in color range

    contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    pointsPrev = points 
    points = list() # empty the array to add new points
    
    for cnt in contours:
        if cv2.contourArea(cnt) > 100:  # only continues if size of the object is large enough (removes noise)
            x1, y1, w, h = cv2.boundingRect(cnt) # finds a bounding box for each object
            cv2.rectangle(frame, (x1, y1), (x1 + w, y1 + h), (0, 0, 255), 2)

            points.append([int(x1 + 0.5*w), int(y1 + 0.5*h)]) # adds center point of bounding to list (for each bbox)
            # points.append([int(x1 + 0.5*w), y1]) # adds bottom-center point of bounding to list (for each bbox) (alternative to above line)
            # need to cast int() because you can't have half a frame

    for i in pointsPrev:
        for j in points:
            if ( np.sqrt( (i[0] - j[0])**2 + (i[1] - j[1])**2 ) < 50 ):     # will draw a line a point from the current frame and a point from the previous frame are close enough        
                cv2.line(firstFrame, (i[0], i[1]), (j[0], j[1]), color, 3)
                break  # break so two lines don't get drawn from the same point
    
    cv2.imshow('webcam', frame)
    cv2.imshow('blur', frame_blur)
    cv2.imshow('mask', mask)
    cv2.imshow('path', firstFrame)
    if cv2.waitKey(40) & 0xFF == ord(' '):
        break  # ends loop when spacebar is pressed

capture.release()
cv2.destroyAllWindows()  # closes window, only reaches here when spacebar is pressed